In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,log_loss
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')

In [79]:
training_df=pd.read_csv("data/optdigits.tra",header=None)
training_df=training_df[((training_df.iloc[:,-1]==1) | (training_df.iloc[:,-1]==7))]
training_df.replace({7:0},inplace=True)
DIMENSIONS=64
models=[]
data_partitions=[]

In [87]:
class LogisticRegression:
    def __init__(self,learning_rate,momentum,dimensions,a,b):
        self.learning_rate=learning_rate
        self.momentum=momentum
        self.dimensions=dimensions
        self.weights=np.random.uniform(low=0,high=0.0001,size=(dimensions,1))
        self.a=a
        self.b=b
    def print_model_parameters(self,i):
        print("*"*45)
        print("Run:{0}\nInitial learning rate:{1}\nInitial momentum:{2}".format(i,self.learning_rate,self.momentum))
    def sigmoid_activation(self,x):
        return 1/(1+np.exp(-x))
    def cross_entropy(self,y_true,y_pred):
        return log_loss(y_true,y_pred)
    def train(self,x_train,y_train,n_iter=50):
        #print(x_train.shape)
        errors=[]
        converged=False
        while converged==False and n_iter!=0:
            error=0
            predictions=[]
            temp=np.zeros(shape=(self.dimensions))
            d_weights=np.zeros(shape=(self.dimensions))
            for i in range(x_train.shape[0]):
                o=0
                for j in range(x_train.shape[1]):
                    o+=(self.weights[j]*x_train[i,j])
                y_pred=self.sigmoid_activation(o)
                for j in range(x_train.shape[1]):
                    #d_weights[j]+=learning_rate*(y_train[i]-y_pred)*x_train[i,j]
                    d_weights[j]=(self.learning_rate*(y_train[i]-y_pred)*x_train[i,j])+(self.momentum*d_weights[j])
                predictions.append(y_train[i]-y_pred)
            isDecreasing=True
            errors.append(self.cross_entropy(y_train,np.array(predictions)))
            current_error=errors[-1]
            for i in range(len(errors)-1):
                if errors[i]<=current_error:
                    isDecreasing=False
                    break
            if isDecreasing:
                self.learning_rate+=self.a
            else:
                self.learning_rate-=(self.b*self.learning_rate)
            for j in range(self.dimensions):
                #self.weights[j]+=self.learning_rate*d_weights[j]
                self.weights[j]+=(d_weights[j])
            try:
                if errors[-2]==errors[-1]:
                    converged=True
            except IndexError:
                continue
        self.evaluate("Training",y_train,self.predict(x_train))
    def predict(self,x):
        y_pred=self.sigmoid_activation(np.sum(x*self.weights.T,axis=1))
        y_pred=np.array(y_pred>0.5,dtype=np.int16)
        return y_pred
    def evaluate(self,string,y_true,y_pred):
        print("{0} error rate for run :{1}".format(string,1-accuracy_score(y_true,y_pred)))

In [88]:
for i in range(10):
    learning_rate=np.random.uniform(low=0.0001,high=0.001)
    momentum=np.random.uniform(low=0.9,high=0.99)
    model=LogisticRegression(learning_rate,momentum,DIMENSIONS,0.0001,0.0002)
    x_train,x_test,y_train,y_test=train_test_split(training_df.iloc[:,:-1].values,training_df.iloc[:,-1].values,train_size=0.8)
    data_partitions.append((x_train,y_train,x_test,y_test))
    model.print_model_parameters(i)
    model.train(x_train,y_train,10)
    models.append(model)
    model.evaluate("Test",y_test,model.predict(x_test))

*********************************************
Run:0
Initial learning rate:0.0008398644580208868
Initial momentum:0.9138697347056899
Training error rate for run :0.008064516129032251
Test error rate for run :0.019230769230769273
*********************************************
Run:1
Initial learning rate:0.0008452216935527667
Initial momentum:0.9856689746642576
Training error rate for run :0.008064516129032251
Test error rate for run :0.0064102564102563875
*********************************************
Run:2
Initial learning rate:0.00023392883275084658
Initial momentum:0.978839173596995
Training error rate for run :0.009677419354838679
Test error rate for run :0.0
*********************************************
Run:3
Initial learning rate:0.0009637599197665233
Initial momentum:0.9645933451821539
Training error rate for run :0.006451612903225823
Test error rate for run :0.012820512820512775
*********************************************
Run:4
Initial learning rate:0.0009481119506721635
Initial 

In [89]:
abs_avg_weights=[]
for model in models:
    abs_avg_weights.append(np.abs(model.weights))
abs_avg_weights=np.array(abs_avg_weights)
abs_avg_weights=abs_avg_weights.sum(axis=0)


In [90]:
after_elimination_10=abs_avg_weights.argsort()[:int(0.9*DIMENSIONS)]
after_elimination_25=abs_avg_weights.argsort()[:int(0.75*DIMENSIONS)]
after_elimination_50=abs_avg_weights.argsort()[:int(0.5*DIMENSIONS)]

In [91]:
for i in range(10):
    learning_rate=np.random.uniform(low=0.0001,high=0.001)
    momentum=np.random.uniform(low=0.9,high=0.99)
    model=LogisticRegression(learning_rate,momentum,after_elimination_10.shape[0],0.0001,0.0002)
    x_train,y_train,x_test,y_test=data_partitions[i]
    model.print_model_parameters(i)
    model.train(x_train[:,after_elimination_10].reshape(x_train.shape[0],after_elimination_10.shape[0]),y_train,10)
    model.evaluate("Test",y_test,model.predict(x_test[:,after_elimination_10].reshape(x_test.shape[0],after_elimination_10.shape[0])))

*********************************************
Run:0
Initial learning rate:0.0007075531229180688
Initial momentum:0.9264075945312545
Training error rate for run :0.5064516129032258
Test error rate for run :0.4807692307692307
*********************************************
Run:1
Initial learning rate:0.0007987230500566974
Initial momentum:0.9093545170359975
Training error rate for run :0.49677419354838714
Test error rate for run :0.5192307692307692
*********************************************
Run:2
Initial learning rate:0.000586255507016697
Initial momentum:0.9263515809096363
Training error rate for run :0.5032258064516129
Test error rate for run :0.4935897435897436
*********************************************
Run:3
Initial learning rate:0.00031429940924970545
Initial momentum:0.9497960688747504
Training error rate for run :0.5080645161290323
Test error rate for run :0.47435897435897434
*********************************************
Run:4
Initial learning rate:0.0009801200946340295
Initia

In [92]:
for i in range(10):
    learning_rate=np.random.uniform(low=0.0001,high=0.001)
    momentum=np.random.uniform(low=0.9,high=0.99)
    model=LogisticRegression(learning_rate,momentum,after_elimination_25.shape[0],0.0001,0.0002)
    x_train,y_train,x_test,y_test=data_partitions[i]
    model.print_model_parameters(i)
    model.train(x_train[:,after_elimination_25].reshape(x_train.shape[0],after_elimination_25.shape[0]),y_train,10)
    model.evaluate("Test",y_test,model.predict(x_test[:,after_elimination_25].reshape(x_test.shape[0],after_elimination_25.shape[0])))

*********************************************
Run:0
Initial learning rate:0.0008950979427267842
Initial momentum:0.9011400969263312
Training error rate for run :0.5064516129032258
Test error rate for run :0.4807692307692307
*********************************************
Run:1
Initial learning rate:0.0008281057030413462
Initial momentum:0.9164715579328562
Training error rate for run :0.49677419354838714
Test error rate for run :0.5192307692307692
*********************************************
Run:2
Initial learning rate:0.0009395872281053469
Initial momentum:0.9659503670838111
Training error rate for run :0.5032258064516129
Test error rate for run :0.4935897435897436
*********************************************
Run:3
Initial learning rate:0.000831395110255498
Initial momentum:0.9177010810157938
Training error rate for run :0.5080645161290323
Test error rate for run :0.47435897435897434
*********************************************
Run:4
Initial learning rate:0.0009705481187962651
Initial

In [93]:
for i in range(10):
    learning_rate=np.random.uniform(low=0.0001,high=0.001)
    momentum=np.random.uniform(low=0.9,high=0.99)
    model=LogisticRegression(learning_rate,momentum,after_elimination_50.shape[0],0.0001,0.0002)
    x_train,y_train,x_test,y_test=data_partitions[i]
    model.print_model_parameters(i)
    model.train(x_train[:,after_elimination_50].reshape(x_train.shape[0],after_elimination_50.shape[0]),y_train,10)
    model.evaluate("Test",y_test,model.predict(x_test[:,after_elimination_50].reshape(x_test.shape[0],after_elimination_50.shape[0])))

*********************************************
Run:0
Initial learning rate:0.0003214352515815618
Initial momentum:0.9149142081649307
Training error rate for run :0.5064516129032258
Test error rate for run :0.4807692307692307
*********************************************
Run:1
Initial learning rate:0.0006939240566520153
Initial momentum:0.9628680546868233
Training error rate for run :0.49677419354838714
Test error rate for run :0.5192307692307692
*********************************************
Run:2
Initial learning rate:0.00043330985852395705
Initial momentum:0.9569898420674453
Training error rate for run :0.5032258064516129
Test error rate for run :0.4935897435897436
*********************************************
Run:3
Initial learning rate:0.0004351017175335713
Initial momentum:0.93655911682541
Training error rate for run :0.5080645161290323
Test error rate for run :0.47435897435897434
*********************************************
Run:4
Initial learning rate:0.000511758637817235
Initial 